<a href="https://colab.research.google.com/github/ajtamayoh/NLP-CIC-WFU-Contribution-to-PARMEX-shared-task-2022/blob/main/Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP-CIC-WFU contribution to PAR-MEX 2022 IberLEF

Here you are the source code for the paper:

### Using Transformers on Noisy vs. Clean Data for Paraphrase Identification in Mexican Spanish

Authors:

Antonio Tamayo (ajtamayo2019@ipn.cic.mx, ajtamayoh@gmail.com)

Diego A. Burgos (burgosda@wfu.edu)

Alexander Gelbulkh (gelbukh@gelbukh.com)

For bugs or questions related to the code, do not hesitate to contact us (Antonio Tamayo: ajtamayoh@gmail.com)

If you use this code please cite our work:

This will be available soon.


# Requirements

To run this code you need to download the dataset (six .csv files) at: https://drive.google.com/drive/folders/1_lmjtkeajie3YcbsA6OKxg0xOWgWP9nI?usp=sharing

Then, you must create a folder called "Datasets" in the root of your Google Drive and load there all the files previously downloaded.

Once the dataset is ready to use, you should [open this notebook in colab](https://colab.research.google.com/github/ajtamayoh/NLP-CIC-WFU-Contribution-to-PARMEX-shared-task-2022/blob/main/Code.ipynb) and save a copy in your drive.

# About the infrastructure

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed May  4 18:43:22 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


### Install the Transformers and Datasets libraries to run this notebook.

In [ ]:
!pip install datasets transformers[sentencepiece]

     |████████████████████████████████| 325 kB 4.3 MB/s 
     |████████████████████████████████| 4.0 MB 51.7 MB/s 
     |████████████████████████████████| 77 kB 1.9 MB/s 
     |████████████████████████████████| 136 kB 51.9 MB/s 
     |████████████████████████████████| 212 kB 89.6 MB/s 
     |████████████████████████████████| 1.1 MB 63.8 MB/s 
     |████████████████████████████████| 127 kB 70.2 MB/s 
     |████████████████████████████████| 144 kB 73.4 MB/s 
     |████████████████████████████████| 271 kB 72.3 MB/s 
     |████████████████████████████████| 94 kB 3.9 MB/s 
     |████████████████████████████████| 880 kB 65.3 MB/s 
     |████████████████████████████████| 596 kB 65.4 MB/s 
     |████████████████████████████████| 6.6 MB 49.8 MB/s 
     |████████████████████████████████| 1.2 MB 59.4 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=210ca89d1e6f45fb5390ec9bb87f3855f494e9c9d547021d553cd04dc7d3b42a
  Stored in directory: /root/.cac

### Connecting to Google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Exploring the data

In [ ]:
import pandas as pd

In [ ]:
#data = pd.read_csv("/content/drive/MyDrive/Datasets/noisy_parmex_train.csv", delimiter=",")
data = pd.read_csv("/content/drive/MyDrive/Datasets/clean_parmex_train.csv", delimiter=",")
data.head()

,id,Text1,Text2,label
0,1,"su nombre proviene de los persas, los cuales e...","el alimento, cuyo nombre proviene de los persa...",1
1,2,"la congelación con deshidratación, uno de los ...","un método de cocción es el sous vide, que mant...",0
2,3,"la empresa de catering caterers bubble, de lon...",el festival de alcachofa ahumada instantánea ...,0
3,4,el sous vide es un método de cocción que manti...,estos se usan para referir a las transformacio...,0
4,5,el cuidado de la piel también es un factor a t...,la dieta general de la gente consiste en carne...,0


In [ ]:
#data_val = pd.read_csv("/content/drive/MyDrive/Datasets/noisy_parmex_validation.csv", delimiter=",")
data_val = pd.read_csv("/content/drive/MyDrive/Datasets/clean_parmex_val.csv", delimiter=",")
data_val.head()

,id,Text1,Text2
0,7602,el evento está lleno de color y misterio en do...,"en una ofrenda, los dulces no es el único come..."
1,7603,el tequila ya era un muy buen negocio y se con...,poco después se volvió fundamental para la pub...
2,7604,¿los alimentos pierden su sabor? ¿cómo se llev...,es cuestionable si los alimentos pierden su sa...
3,7605,"a pesar de la aventura y las delicias, varios ...",aun cuando el método de sumergir en agua fue e...
4,7606,pero el tronco se cocina en vertical para ahor...,hay una gran diversidad de salsas que varían e...


## Loading the data

In [ ]:
from datasets import load_dataset, DatasetDict
#dataset = load_dataset('csv', data_files="/content/drive/MyDrive/Datasets/noisy_parmex_train.csv", split='train')
dataset = load_dataset('csv', data_files="/content/drive/MyDrive/Datasets/clean_parmex_train.csv", split='train')
train_test = dataset.train_test_split()
test_val = train_test['test'].train_test_split()
raw_datasets = DatasetDict({
    'train': train_test['train'],
    'validation': test_val['train'],
    'test': test_val['test']
    })

Using custom data configuration default-9586c890888d4a00
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-9586c890888d4a00/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)
Loading cached split indices for dataset at /root/.cache/huggingface/datasets/csv/default-9586c890888d4a00/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-b14cab2ce00769bc.arrow and /root/.cache/huggingface/datasets/csv/default-9586c890888d4a00/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-b0bb084cc474d108.arrow
Loading cached split indices for dataset at /root/.cache/huggingface/datasets/csv/default-9586c890888d4a00/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-bdf13519d2cae7a9.arrow and /root/.cache/huggingface/datasets/csv/default-9586c890888d4a00/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-3024171ec3763267.arrow


In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'Text1', 'Text2', 'label'],
        num_rows: 5536
    })
    validation: Dataset({
        features: ['id', 'Text1', 'Text2', 'label'],
        num_rows: 1384
    })
    test: Dataset({
        features: ['id', 'Text1', 'Text2', 'label'],
        num_rows: 462
    })
})

In [ ]:
raw_datasets["train"][0]

{'Text1': 'es muy posible que poco a poco se dieran cuenta que el arroz del proceso de fermentación era igualmente comestible, pero un sabor ácido y con un fuerte olor a pescado.',
 'Text2': 'la nueva forma de comer así pescado crudo con arroz le llamaron sushi rápido, pues era simple y mucho más rápido prepararlo.',
 'id': 1450,
 'label': 0}

### Loading BERTIN and its tokenizer

In [ ]:
#from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

checkpoint = "bertin-project/bertin-roberta-base-spanish"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

loading file https://huggingface.co/bertin-project/bertin-roberta-base-spanish/resolve/main/vocab.json from cache at /root/.cache/huggingface/transformers/f47efb87887425ef9a4ef795bfaa907d57ac9a650d733c7ca621b9eced3235e8.0b968504b15b2f5e6e9e491723f31987782cb976064aff9a7c6188cc2d5ce8bc
loading file https://huggingface.co/bertin-project/bertin-roberta-base-spanish/resolve/main/merges.txt from cache at /root/.cache/huggingface/transformers/aba9e0895dea47dd4208a36012ffd3eb21eb4c5f7ce0be6547afb37cdd4ddef4.0d24ae8bd5fabb1f5020f91bc602cefeb5a2938ab77e21769d28776345634b23
loading file https://huggingface.co/bertin-project/bertin-roberta-base-spanish/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/baad57d0f574d3e660cafb14601d0ecebe83f25071d59f3e51d225d75285b773.23862d4cf295978aaf1f42b721fda2d8ece45b112d21f0c83905e5b05454e440
loading file https://huggingface.co/bertin-project/bertin-roberta-base-spanish/resolve/main/added_tokens.json from cache at None
loading file

### Tokenization

In [ ]:
def tokenize_function(example):
    return tokenizer(example["Text1"], example["Text2"], truncation=True)

In [ ]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-9586c890888d4a00/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-83fb6828f683d0d9.arrow


  0%|          | 0/2 [00:00<?, ?ba/s]

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-9586c890888d4a00/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-2b498274a4c5adc0.arrow


In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'Text1', 'Text2', 'label', 'input_ids', 'attention_mask'],
        num_rows: 5536
    })
    validation: Dataset({
        features: ['id', 'Text1', 'Text2', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1384
    })
    test: Dataset({
        features: ['id', 'Text1', 'Text2', 'label', 'input_ids', 'attention_mask'],
        num_rows: 462
    })
})

### Setting hyperparameters

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments("trainer-test", num_train_epochs=7, learning_rate=5e-7) #You can change "trainer-test" for a path in your Gdrive if you want to save the model.

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


### Loading the model's architecture and changing the head of prediction

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

loading configuration file https://huggingface.co/bertin-project/bertin-roberta-base-spanish/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/aa24361f0b7bed62876f6cd0a784a2b622c1959523906d89eeb1112139a4864a.604375914a974f33506fc8902fe2d195407cbf2316166e695267dd5655a736b3
Model config RobertaConfig {
  "_name_or_path": "bertin-project/bertin-roberta-base-spanish",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.18.0",
  "type_vocab_size": 1,
  "use_ca

### Prepairing the training

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

### Fine-tuning BERTIN for paraphrasing identification

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: Text1, Text2, id. If Text1, Text2, id are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 5536
  Num Epochs = 7
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 4844


Step,Training Loss
500,0.417700
1000,0.263600
1500,0.222600
2000,0.134600
2500,0.103400
3000,0.084200
3500,0.077400
4000,0.075100
4500,0.073300


Saving model checkpoint to trainer-test/checkpoint-500
Configuration saved in trainer-test/checkpoint-500/config.json
Model weights saved in trainer-test/checkpoint-500/pytorch_model.bin
tokenizer config file saved in trainer-test/checkpoint-500/tokenizer_config.json
Special tokens file saved in trainer-test/checkpoint-500/special_tokens_map.json
Saving model checkpoint to trainer-test/checkpoint-1000
Configuration saved in trainer-test/checkpoint-1000/config.json
Model weights saved in trainer-test/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in trainer-test/checkpoint-1000/tokenizer_config.json
Special tokens file saved in trainer-test/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to trainer-test/checkpoint-1500
Configuration saved in trainer-test/checkpoint-1500/config.json
Model weights saved in trainer-test/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in trainer-test/checkpoint-1500/tokenizer_config.json
Special tokens file saved

TrainOutput(global_step=4844, training_loss=0.15408573812142962, metrics={'train_runtime': 816.2048, 'train_samples_per_second': 47.478, 'train_steps_per_second': 5.935, 'total_flos': 1715015414379840.0, 'train_loss': 0.15408573812142962, 'epoch': 7.0})

### Validating the model fune-tuned

In [ ]:
val_predictions = trainer.predict(tokenized_datasets["validation"])
test_predictions = trainer.predict(tokenized_datasets["test"])
print(val_predictions.predictions.shape, predictions.label_ids.shape)
print(test_predictions.predictions.shape, predictions.label_ids.shape)

The following columns in the test set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: Text1, Text2, id. If Text1, Text2, id are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 462
  Batch size = 8


(462, 2) (462,)


In [ ]:
import numpy as np

val_preds = np.argmax(val_predictions.predictions, axis=-1)
test_preds = np.argmax(test_predictions.predictions, axis=-1)

In [ ]:
from datasets import load_metric

metric = load_metric("glue", "mrpc")
metric.compute(predictions=val_preds, references=predictions.label_ids)

{'accuracy': 0.9545454545454546, 'f1': 0.8679245283018868}

In [ ]:
metric.compute(predictions=test_preds, references=predictions.label_ids)

In [ ]:
#Measures for validation
def metricas(y_true,y_pred):
  from sklearn.metrics import f1_score, recall_score, precision_score
  print("f1=", f1_score(y_true, y_pred, average='binary'))
  print("Recall=", recall_score(y_true, y_pred, average='binary'))
  print("Precision=", precision_score(y_true, y_pred, average='binary'))

In [ ]:
#Validation
metricas(raw_datasets["validation"][:]['label'], val_preds)

f1= 0.9174311926605504
Recall= 0.9090909090909091
Precision= 0.9259259259259259


In [ ]:
#Test
metricas(raw_datasets["test"][:]['label'], test_preds)

f1= 0.8679245283018868
Recall= 0.8734177215189873
Precision= 0.8625


In [ ]:
j = 0
b = 0
for i in range(preds.shape[0]):
  #if raw_datasets["validation"][i]['label'] == 1:
  if raw_datasets["test"][i]['label'] == 1:
    j+=1
    #print(raw_datasets["test"][i]['label'], '\t', preds[i])
    #if raw_datasets["validation"][i]['label'] == preds[i]:
    if raw_datasets["test"][i]['label'] == preds[i]:
      b+=1
print(b,"corrects, of ", j)
print("Accuracy over positive samples: ", b/j)  

75 correctas, de  76
Accuracy sobre muestras positivas:  0.9868421052631579


In [ ]:
from datasets import load_dataset, DatasetDict

#dataset = load_dataset('csv', data_files="/content/drive/MyDrive/Datasets/noisy_parmex_validation.csv")
dataset = load_dataset('csv', data_files="/content/drive/MyDrive/Datasets/clean_parmex_val.csv")
val_dataset = DatasetDict({
    'val': dataset['train']
    })

Using custom data configuration default-5aaa32897a8e9226
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-5aaa32897a8e9226/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
val_dataset["val"][0]

{'Text1': 'el evento está lleno de color y misterio en donde cantos y rezos acompañan al espíritu a degustar la comida.',
 'Text2': 'en una ofrenda, los dulces no es el único comestible también se pone comida que era de agrado del fallecido.',
 'id': 7602}

In [ ]:
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-5aaa32897a8e9226/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-d34cbacf141cb74b.arrow


In [ ]:
predictions = trainer.predict(tokenized_val_dataset["val"])
print(predictions.predictions.shape)

The following columns in the test set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: Text1, id, Text2. If Text1, id, Text2 are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 97
  Batch size = 8


(97, 2)


In [ ]:
val_preds = np.argmax(predictions.predictions, axis=-1)

## Formatting the output

In [ ]:
def formatting(val_preds, output_file):
  f = open(output_file, "w", newline='')
  i = 1
  for p in val_preds:
      if p == 1:
          f.write('"ParaphraseIdentification"\t"'+str(i)+'"\t"P"\n')
      else:
          f.write('"ParaphraseIdentification"\t"'+str(i)+'"\t"NP"\n')
      i+=1
  f.close()

In [ ]:
formatting(val_preds, "/content/drive/MyDrive/Results.txt")

## Save and load the fine-tuned model to inference

In [ ]:
#Saving the model
#Uncomment the next line if you want to save the model in your Gdrive
trainer.save_model("/content/drive/MyDrive/NLP-CIC-WFU-BERTIN-Fine-Tuned-for-Paraphrase-Identification")


Saving model checkpoint to /content/drive/MyDrive/Doctorado CIC-IPN/ParMex/Models/Models_tokenizers/mrm8488_longformer-base-4096-spanish__5_epochs_lr_5e-7_Trained_with_new_dataset_V9
Configuration saved in /content/drive/MyDrive/Doctorado CIC-IPN/ParMex/Models/Models_tokenizers/mrm8488_longformer-base-4096-spanish__5_epochs_lr_5e-7_Trained_with_new_dataset_V9/config.json
Model weights saved in /content/drive/MyDrive/Doctorado CIC-IPN/ParMex/Models/Models_tokenizers/mrm8488_longformer-base-4096-spanish__5_epochs_lr_5e-7_Trained_with_new_dataset_V9/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/Doctorado CIC-IPN/ParMex/Models/Models_tokenizers/mrm8488_longformer-base-4096-spanish__5_epochs_lr_5e-7_Trained_with_new_dataset_V9/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/Doctorado CIC-IPN/ParMex/Models/Models_tokenizers/mrm8488_longformer-base-4096-spanish__5_epochs_lr_5e-7_Trained_with_new_dataset_V9/special_tokens_map.json


In [ ]:
#Loading the model
from transformers import AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding

model_loaded = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/NLP-CIC-WFU-BERTIN-Fine-Tuned-for-Paraphrase-Identification", num_labels=2)
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/NLP-CIC-WFU-BERTIN-Fine-Tuned-for-Paraphrase-Identification")


## Inference on training dataset

In [ ]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:

import torch
import numpy as np
preds = []
for i in range(tokenized_datasets.shape['validation'][0]):
#for i in range(tokenized_datasets.shape['test'][0]):
  sample = torch.tensor([tokenized_datasets["validation"][i]['input_ids']])
  #sample = torch.tensor([tokenized_datasets["test"][i]['input_ids']])
  predictions = model_loaded(sample)
  preds.append(np.argmax(predictions.logits[0].detach().cpu().numpy()))


In [ ]:
#For validation
metricas(raw_datasets["validation"][:]['label'], preds)

f1= 0.9611451942740286
Recall= 0.94
Precision= 0.9832635983263598


In [ ]:
#For test
metricas(raw_datasets["test"][:]['label'], preds)

f1= 0.9655172413793104
Recall= 0.958904109589041
Precision= 0.9722222222222222


## Inference on validation dataset

In [ ]:
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

  0%|          | 0/1 [00:00<?, ?ba/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:

import torch
import numpy as np
val_preds = []
for i in range(tokenized_val_dataset.shape['val'][0]):
  sample = torch.tensor([tokenized_val_dataset["val"][i]['input_ids']])
  predictions = model_loaded(sample)
  val_preds.append(np.argmax(predictions.logits[0].detach().cpu().numpy()))


In [ ]:
len(val_preds)

97

In [ ]:
formatting(val_preds, "/content/drive/MyDrive/results_val.txt")

## Inference on test dataset

In [ ]:

from datasets import load_dataset, DatasetDict

#dataset = load_dataset('csv', data_files="/content/drive/MyDrive/Datasets/noisy_parmex_test.csv")
dataset = load_dataset('csv', data_files="/content/drive/MyDrive/Datasets/clean_parmex_test.csv")
test_dataset = DatasetDict({
    'test': dataset['train']
    })


Using custom data configuration default-4666d43810f7adaf
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-4666d43810f7adaf/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
test_dataset['test'][0]

{'Text1': 'el evento está lleno de color y misterio en donde cantos y rezos acompañan al espíritu a degustar la comida.',
 'Text2': 'no sólo los dulces de un altar se comen, pues también se colocan alimentos que le gustaban al muerto.',
 'id': 7702}

In [ ]:
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

  0%|          | 0/3 [00:00<?, ?ba/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:

import torch
import numpy as np
test_preds = []
for i in range(tokenized_test_dataset.shape['test'][0]):
  sample = torch.tensor([tokenized_test_dataset["test"][i]['input_ids']])

  #Model saved and loaded
  predictions = model_loaded(sample)

  test_preds.append(np.argmax(predictions.logits[0].detach().cpu().numpy()))


In [ ]:
len(test_preds)

2819

In [ ]:
#test_preds

In [ ]:
formatting(test_preds, "/content/drive/MyDrive/New_Test_predictions_NLP-CIC-WFU-BERTIN-for-Paraphrase-identification.txt")